In [2]:
# Importing OpenCV library for computer vision tasks
import cv2

# Importing NumPy for numerical operations and array manipulations
import numpy as np

# Importing time to add a delay before the cloak effect starts
import time

# Starting the video capture from the default webcam (device index 0)
cap = cv2.VideoCapture(0)

# Giving the camera time to warm up (for better background capture)
time.sleep(2)

# Capturing the static background in a few frames
# This will be used to replace the cloak region
background = 0

# Capture multiple frames to reduce noise and get a clear background
for i in range(30):
    ret, background = cap.read()

# Flipping the background image horizontally
# (Because webcam shows mirrored image; flipping makes movement natural)
background = np.flip(background, axis=1)

# Start capturing video frame-by-frame
while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        break

    # Flip the current frame to match the flipped background
    img = np.flip(img, axis=1)

    # Convert the image from BGR (default OpenCV color) to HSV color space
    # HSV is better for color detection
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Define the range of red color in HSV
    # Red has two ranges in HSV due to the circular nature of hue
    lower_red1 = np.array([0, 120, 50])
    upper_red1 = np.array([10, 255, 255])
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)  # Create mask for red part 1

    lower_red2 = np.array([170, 120, 70])
    upper_red2 = np.array([180, 255, 255])
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)  # Create mask for red part 2

    # Combine both masks to cover full red range
    mask = mask1 + mask2

    # Refine the mask using morphological operations (remove noise)
    # Morph open: remove small white spots
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN,
                            np.ones((3, 3), np.uint8), iterations=2)

    #  increase white region to merge nearby white areas
    mask = cv2.dilate(mask, np.ones((3, 3), np.uint8), iterations=1)

    # Invert mask to get parts that are not red
    mask_inv = cv2.bitwise_not(mask)

    # Segment out the cloak area from the background image
    cloak_area = cv2.bitwise_and(background, background, mask=mask)

    # Segment out the non-cloak area from the current frame
    non_cloak_area = cv2.bitwise_and(img, img, mask=mask_inv)

    # Combine both cloak and non-cloak parts
    final_output = cv2.addWeighted(cloak_area, 1, non_cloak_area, 1, 0)

    # Display the final output frame
    cv2.imshow("Harry Potter's Invisible Cloak", final_output)

    # Wait for 1 ms and break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object
cap.release()

# Destroy all OpenCV windows
cv2.destroyAllWindows()